In [ ]:
import sys

!{sys.executable} -m pip install ijson tabulate

In [ ]:
import copy
import datetime
import hpq
import html
import ijson
import IPython.display
import tabulate

date = "2022-11-29"
connection = hpq.create_web_socket_client()
spreads = list(
    filter(
        lambda obj: "legs" in obj.keys(),
        ijson.items(
            connection.stream(
                {
                    "query": "products",
                    "source": ["sgx"],
                    "date": date,
                    "type": "metadata",
                }
            ),
            "item",
        ),
    )
)


def get_trade():
    for spread in spreads:
        for obj in filter(
            lambda obj: obj["type"] != "no_day",
            ijson.items(
                connection.stream(
                    {
                        "query": "ticks",
                        "source": "sgx",
                        "date": date,
                        "start_time": "00:00:00",
                        "end_time": "23:59:59.999999999",
                        "messages": ["TRD"],
                        "product": str(spread["product_id"]),
                    }
                ),
                "item",
            ),
        ):
            connection.cancel()
            return (spread, obj)
    return None


(spread, trade) = get_trade()


def date(ts):
    return datetime.datetime.utcfromtimestamp(ts / 1000000000).strftime("%Y-%m-%d")


def time(ts):
    retr = datetime.datetime.utcfromtimestamp(ts / 1000000000).strftime("%H:%M:%S")
    ns = str(ts % 1000000000)
    while len(ns) != 9:
        ns = "0" + ns
    return f"{retr}.{ns}"


def get_outright_trades(product_id):
    begin = trade["exchange_timestamp"]
    end = begin + 1
    return filter(
        lambda obj: obj["combo_group_id"] == trade["combo_group_id"],
        ijson.items(
            connection.stream(
                {
                    "query": "ticks",
                    "source": "sgx",
                    "start_date": date(begin),
                    "end_date": date(end),
                    "start_time": time(begin),
                    "end_time": time(end),
                    "messages": ["TRD"],
                    "timestamp_mode": "exchange",
                    "time_zone": "UTC",
                    "product": str(product_id),
                }
            ),
            "item",
        ),
    )


name = html.escape(spread["name"])
IPython.display.display(IPython.display.HTML(f"<h1>{name}</h1>"))
IPython.display.display(IPython.display.HTML(f"<h2>Legs</h2>"))
table = tabulate.tabulate(spread["legs"], tablefmt="html", headers="keys")
IPython.display.display(IPython.display.HTML(table))
IPython.display.display(IPython.display.HTML(f"<h2>Metadata</h2>"))


def metadata():
    retr = copy.deepcopy(spread)
    del retr["legs"]
    yield retr
    when = trade["exchange_timestamp"] + 1
    for leg in spread["legs"]:
        for obj in ijson.items(
            connection.stream(
                {
                    "query": "snapshot",
                    "source": "sgx",
                    "date": date(when),
                    "time": time(when),
                    "type": "productinfo",
                    "timestamp_mode": "exchange",
                    "time_zone": "UTC",
                    "product": str(leg["product_id"]),
                }
            ),
            "item",
        ):
            yield obj


table = tabulate.tabulate(
    list(map(hpq.format, metadata())), tablefmt="html", headers="keys"
)
IPython.display.display(IPython.display.HTML(table))
IPython.display.display(
    IPython.display.HTML(f"<h2>All Trades for Selected Execution</h2>")
)


def get_trades():
    global trade
    yield copy.deepcopy(trade)
    for leg in spread["legs"]:
        for obj in get_outright_trades(leg["product_id"]):
            yield obj


table = tabulate.tabulate(
    list(map(hpq.format, get_trades())), tablefmt="html", headers="keys"
)
IPython.display.display(IPython.display.HTML(table))